## NeuralProphet
Reference: \\
https://neuralprophet.com/contents.html

In [ ]:
!pip3 install neuralprophet==0.4.2

In [ ]:
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")
from neuralprophet import set_random_seed
set_random_seed(0)
import pandas as pd
import numpy as np
import plotly.express as px


### Office

In [ ]:
office_csv_path     = '/content/drive/MyDrive/Interpretable Forecasting of Buildings Energy Consumption/code/outputs/data/office/final_office.csv'
rawData = pd.read_csv(office_csv_path)

In [ ]:
rawData.head()

In [ ]:
# office
data = pd.DataFrame({"ds"             : rawData["date"],
                     "y"              : rawData["energy_south_total"],
                     "dew_temp"       : rawData["dew_point_temperature_set_1d"],
                     "humidity"       : rawData["relative_humidity_set_1"],
                     "solar_radiation": rawData["solar_radiation_set_1"],
                     "out_temp"       : rawData["air_temp_set_1_2"],
                     "in_temp"        : rawData["internal_temp"],
                     "occ"            : rawData["occ_south_total"],
                     "month"          : rawData["month"]
                     })

trainEndBoundary = '2019-02-03'
validStartBoundary = '2019-02-03'
validEndBoundary = '2019-02-10'
testStartBoundary = '2019-02-10'
testEndBoundary = '2019-02-21'
index = data['ds']
train = data.loc[index <= trainEndBoundary]
valid = data.loc[(index >= validStartBoundary) & (index <= validEndBoundary)]
test  = data.loc[(index >= testStartBoundary) & (index <= testEndBoundary)]

train_valid = pd.concat([train, valid])
trainData = train_valid
# trainData = train

# valid_test = pd.concat([valid, test])
# testData = valid_test
testData = test

testData['ds'] = testData['ds'].astype('datetime64[ns]')
trainData['ds'] = trainData['ds'].astype('datetime64[ns]')

lagsNum = 3*4*24
m = NeuralProphet(
    n_forecasts=12,
    n_lags=lagsNum,
    num_hidden_layers=1,
    normalize = 'standardize',
    drop_missing=True
)

m = m.add_lagged_regressor(names=["occ", "in_temp", "out_temp", "solar_radiation", "humidity", "dew_temp"])
m = m.add_future_regressor(name="month")
m = m.highlight_nth_step_ahead_of_each_forecast(1)

WARNING - (py.warnings._showwarnmsg) - <ipython-input-3-1f7c50cc8ad1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['ds'] = testData['ds'].astype('datetime64[ns]')

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['ds'] = testData['ds'].astype('datetime64[ns]')



In [ ]:
print(train.iloc[-1]['ds'])
print(valid.iloc[0]['ds'])
print(valid.iloc[-1]['ds'])
print(test.iloc[0]['ds'])
print(test.iloc[-1]['ds'])

2019-02-02 23:45:00
2019-02-03 00:00:00
2019-02-09 23:45:00
2019-02-10 00:00:00
2019-02-20 23:45:00


In [ ]:
metrics_train = m.fit(df=trainData, freq="15min")

  0%|          | 0/158 [00:00<?, ?it/s]

  0%|          | 0/158 [00:00<?, ?it/s]

### RENergetic

In [ ]:
renergetic_csv_path = '/content/drive/MyDrive/Interpretable Forecasting of Buildings Energy Consumption/code/outputs/data/renergetic/debit2_2020_2021.csv'
renergetic_NoOutlier_csv_path = '/content/drive/MyDrive/Interpretable Forecasting of Buildings Energy Consumption/code/outputs/data/renergetic/debit2_2020_2021_NoOutlier.csv'
rawData_NoOutlier = pd.read_csv(renergetic_NoOutlier_csv_path)
rawData = pd.read_csv(renergetic_csv_path)

In [ ]:
# renergetic
data = pd.DataFrame({"ds":       rawData["timestamp"],
                     "y":        rawData["MW"],
                     "m3_h":     rawData["m3_h"],
                     "TEMP_OUT": rawData["TEMP_OUT"],
                     "day_of_week":    rawData["day_of_week"]
                     })

data_NoOutlier = pd.DataFrame({"ds":       rawData_NoOutlier["timestamp"],
                                "y":        rawData_NoOutlier["MW"],
                                "m3_h":     rawData_NoOutlier["m3_h"],
                                "TEMP_OUT": rawData_NoOutlier["TEMP_OUT"],
                                "day_of_week":    rawData_NoOutlier["day_of_week"]
                                })

trainEndBoundary = '2021-11-01'
validStartBoundary = '2021-11-01'
validEndBoundary = '2021-12-01'
testStartBoundary = '2021-12-01'
testEndBoundary = '2021-12-31'
index = data['ds']
train = data.loc[index <= trainEndBoundary]
valid = data_NoOutlier.loc[(index >= validStartBoundary) & (index <= validEndBoundary)]
# valid = data.loc[(index >= validStartBoundary) & (index <= validEndBoundary)]
test  = data_NoOutlier.loc[(index >= testStartBoundary) & (index <= testEndBoundary)]
# test  = data.loc[(index >= testStartBoundary) & (index <= testEndBoundary)]

train_valid = pd.concat([train, valid])
trainData = train_valid
# trainData = train

# valid_test = pd.concat([valid, test])
# testData = valid_test
# testData = valid
testData = test

testData['ds'] = testData['ds'].astype('datetime64[ns]')
testData['ds'] = testData['ds'].dt.round("H")
trainData['ds'] = trainData['ds'].astype('datetime64[ns]')
trainData['ds'] = trainData['ds'].dt.round("H")

lagsNum = 3*24
m = NeuralProphet(
    n_forecasts=12,
    n_lags=lagsNum,
    # num_hidden_layers=1,
    normalize = 'standardize',
    drop_missing=True
)
m = m.add_lagged_regressor(names=['m3_h', 'TEMP_OUT'])
m = m.add_future_regressor(name='day_of_week')
m = m.highlight_nth_step_ahead_of_each_forecast(1)


WARNING - (py.warnings._showwarnmsg) - <ipython-input-3-9926979bfa5b>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['ds'] = testData['ds'].astype('datetime64[ns]')

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData['ds'] = testData['ds'].astype('datetime64[ns]')

WARNING - (py.warnings._showwarnmsg) - <ipython-input-3-9926979bfa5b>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [ ]:
print(train.iloc[0]['ds'])
print(train.iloc[-1]['ds'])
print(valid.iloc[0]['ds'])
print(valid.iloc[-1]['ds'])
print(test.iloc[0]['ds'])
print(test.iloc[-1]['ds'])

2020-08-15 00:00:00
2021-10-31 22:59:59.971
2021-11-01 00:00:00.000
2021-11-30 22:59:59.971
2021-12-01 00:00:00
2021-12-30 23:00:00


In [ ]:
metrics_train = m.fit(df=trainData, freq="H")

Finding best initial lr:   0%|          | 0/251 [00:00<?, ?it/s]

### Prediction

In [ ]:
def forecastWithNP(testData, forecastStartBoundary=None, forecastHorizonStr='yhat1'):

  forecast = m.predict(df=testData)
  forecMod = forecast.dropna()
  testDMod = testData.copy()
  testDMod.set_index('ds', inplace=True)
  forecMod.set_index('ds', inplace=True)
  predIdx = forecMod.index
  obsIdx  = testDMod.index

  if forecastStartBoundary == None: forecastStartBoundary = testDMod.index[0]
  forecastEndBoundary = testDMod.index[-1]
  predSeries = forecMod[forecastHorizonStr].loc[(predIdx >= forecastStartBoundary) & (predIdx <= forecastEndBoundary)].astype('float32')
  gtSeries   = testDMod['y'].loc[(obsIdx >= forecastStartBoundary) & (obsIdx <= forecastEndBoundary)].astype('float32')
  gtPredDf   = pd.concat([gtSeries, predSeries], axis=1)
  gtPredDf = gtPredDf.dropna()

  resultsDict = {}
  resultsDict['obs']  = gtPredDf['y']
  resultsDict['pred'] = gtPredDf[forecastHorizonStr]

  return resultsDict

In [ ]:
#SMAPE - from  'https://vedexcel.com/how-to-calculate-smape-in-python/'
def smape(obs, pred):
  obs = np.array(obs)
  pred = np.array(pred)
  return 100/len(obs) * np.sum(2 * np.abs(pred - obs) / (np.abs(obs) + np.abs(pred)))

In [ ]:
# forecastStartBoundary = testDMod.index[0]
# forecastStartBoundary = '2021-12-09'
# forecastStartBoundary = '2019-02-12'
# testResultsDict = forecastWithNP(testData, forecastStartBoundary)
testResultsDict = forecastWithNP(testData.copy())

In [ ]:
figDict = {}
figDict['predicted electricity'] = testResultsDict['pred']
figDict['observed electricity']  = testResultsDict['obs']
figDict['time']                  = testResultsDict['obs'].index

print("sMAPE Measure", smape(figDict['observed electricity'], figDict['predicted electricity']))

fig = px.line(figDict, x=figDict['time'], y=["observed electricity","predicted electricity"])
fig.show()

sMAPE Measure 36.742763161339646


### Components


In [ ]:
forecast = m.predict(df=testData)
m.plot_components(forecast)

In [ ]:
m.plot_parameters()

In [ ]:
# fit a model on training data and evaluate on validation set.
metrics_train1 = m.fit(df=train, freq="H")
metrics_val = m.test(df=valid)
metrics_test = m.test(df=test)

# # refit model on training and validation data and evaluate on test set.
# metrics_train2 = m.fit(df=df_train_val, freq="H")
# metrics_test = m.test(df=df_test)

  0%|          | 0/263 [00:00<?, ?it/s]

  0%|          | 0/263 [00:00<?, ?it/s]

  0%|          | 0/263 [00:00<?, ?it/s]

In [ ]:
# forecast = m.predict(valid)
# fig = m.plot(forecast)
# comp = m.plot_components(forecast)
# # param = m.plot_parameters()